In [1]:
import sys
sys.path.append("../../module")
from nxrd.Compound import Compound
from classes import MCS_Finder
import kcf.converter as kcfco
from rdkit import Chem

In [2]:
f = MCS_Finder("Streptomyces")
Cnlist = f.get_Cnlist_from_label2("C(C)-C(C)-C(C-C-C-C-C-C)-C-C-C-C-C-C-C-C-C")

In [3]:
complist = []
for i in Cnlist:
    complist.append(Compound())
print(complist)

[<nxrd.Compound.Compound object at 0x10f3e7908>, <nxrd.Compound.Compound object at 0x10f3e7978>, <nxrd.Compound.Compound object at 0x10f3e79e8>, <nxrd.Compound.Compound object at 0x10f3e7a58>, <nxrd.Compound.Compound object at 0x10f3e7ac8>, <nxrd.Compound.Compound object at 0x10f3e7b38>, <nxrd.Compound.Compound object at 0x10f3e7ba8>, <nxrd.Compound.Compound object at 0x10f3e7c18>, <nxrd.Compound.Compound object at 0x10f3e7c88>, <nxrd.Compound.Compound object at 0x10f3e7cf8>, <nxrd.Compound.Compound object at 0x10f3e7d68>, <nxrd.Compound.Compound object at 0x10f3e7dd8>, <nxrd.Compound.Compound object at 0x10f3e7e48>, <nxrd.Compound.Compound object at 0x10f3e7eb8>, <nxrd.Compound.Compound object at 0x10f3e7f28>, <nxrd.Compound.Compound object at 0x10f3e7f98>, <nxrd.Compound.Compound object at 0x10ed784a8>, <nxrd.Compound.Compound object at 0x10ed78518>, <nxrd.Compound.Compound object at 0x10ed784e0>, <nxrd.Compound.Compound object at 0x10f3f9080>]


In [4]:
pagerange = []
for i in range(1, 52):
    page = str(i)
    with open("../../../database/knapsack-kcf/KNApSAck" + page + ".kcf")as f:
        clist = f.read().split()
        pagerange.append(clist[1])
print(pagerange)

['C00000001', 'C00001119', 'C00002144', 'C00003163', 'C00004172', 'C00005175', 'C00006180', 'C00007188', 'C00008214', 'C00009218', 'C00010225', 'C00011238', 'C00012247', 'C00013252', 'C00014261', 'C00015266', 'C00016267', 'C00017271', 'C00018274', 'C00019276', 'C00020285', 'C00021288', 'C00022289', 'C00023292', 'C00024299', 'C00025305', 'C00026310', 'C00027314', 'C00028321', 'C00029327', 'C00030330', 'C00031343', 'C00032344', 'C00033345', 'C00034347', 'C00035347', 'C00036350', 'C00037352', 'C00038354', 'C00039356', 'C00040358', 'C00041361', 'C00042362', 'C00043363', 'C00044366', 'C00045369', 'C00046369', 'C00047370', 'C00048371', 'C00049371', 'C00050371']


In [5]:
counter = 0
for z, i in enumerate(Cnlist):
    num = int(i[1:])
    for p, k in enumerate(pagerange[1:]):
        k2 = int(k[1:])
        if k2 > num:
            k3 = str(p+1)
            with open("../../../database/knapsack-kcf/KNApSAck" + k3 + ".kcf")as f:
                Clist = f.read().split("///\n")
                try:
                    for C in Clist:
                        if i == C.split()[1]:
                            molblock = kcfco.kcf_to_molblock(C)
                            # print("OK", i)
                            # print(molblock[1])
                            mol = Chem.MolFromMolBlock(molblock[1])
                            if mol is None:
                                print("None", i, z, k3)
                                if "#+" in C or "#-" in C:
                                    print("Charge in\n")
                                counter += 1
                                break
                            complist[z].input_rdkmol(mol)
                            if "#+" in C or "#-" in C:
                                print(i, z, k3, "Charge in\n")
                            break
                except IndexError:
                    counter += 1
                    print("DAME", i, z)
            break
print(counter)

0


In [6]:
import time
start = time.time()
veclist = [0] * len(Cnlist)
for i in range(len(Cnlist)):
    veclist[i] = kcfco.kcf_vec(complist[i])
    print(time.time() - start)
print("finish")

5.574361085891724
11.47358512878418
15.753422975540161
20.385885000228882
25.96662402153015
28.121263027191162
33.3217351436615
38.30070209503174
43.05122494697571
47.955446004867554
52.75716304779053
54.27782607078552
57.638684034347534
62.19811201095581
67.1122989654541
72.56640601158142
78.3229079246521
81.16751909255981
84.25438594818115
88.99604892730713
finish


In [7]:
temp1 = list(veclist[0].pandas()["str"])
strlist = [] * len(Cnlist)
for vec in veclist:
    strlist.append(vec.pandas()["str"])

In [8]:
kyoutuu = []
for vecstr in strlist[0]:
    for vec in strlist:
        if vecstr not in vec.values:
            break
    else:
        kyoutuu.append(vecstr)

In [9]:
print(len(kyoutuu))

159


In [10]:
import re
temp2 = []
for i in kyoutuu:
    temp3 = i.split("-")
    length = len(temp3)
    for i in re.findall("-[0-9]", "-".join(temp3)):
        length -= 1
    temp2.append((length, temp3))

In [11]:
for group in temp2:
    if group[0] > 6:
        print("-".join(group[1]), group[0])

C-C-C-C-C-C-C-C-C-C,7-C-C-10 12
C-C-C-C-C-C-C-C-C-C-C-N 12
C-C-C-C-C-C-C-C-C,6-C-C-9 11
C-C-C-C-C-C,5-C 7
C-C-C-C-C-C-C-C,7-C 9
C-C-C-C-C-C-C,6-C 8
C-C-C-C-C-C-C-C-N,1-6,1-9 9
C-C-C-C-C-C-C-C-C-C-C,8-C-C-11 13
C-C-C-C-C,4-C,3-C-C 8
C-C-C-C-C-C-C-C-C-C-C-C,9-C-C-12 14
C8-C8-C8-C8-C8-C8-C8-C8-N4,1-6,1-9 9
C-C-C-C-C-C-C-C,5-C-C-8 10
C-N-C-C-C-C-C-C-C-C-C-C,9-C-C-12 14
N-C-C-C-C-C-C-C-C-C-C,8-C-C-11 13
C-C-C-C-C-C-C-C-C,7-C-C 11
C-C-C-C-C-C-C-C 8
C-C-C-C-C-C-C-C,6-C-C 10
C-C-C-C-C,3-C-C,6-C,4-N 9
C-N-C-C-C-C-C,6-C,5-C-C,9-C 11
O-C-C-C-C,4-C,3-C-C,7-C 9
C-N-C-C-C-C-C-C-C,8-C 10
C-N-C-C-C-C,5-C 7
N-C-C-C-C-C-C-C,7-C 9
C-C-C-C-C-C-C-N,2-7 8
C-C-C-C-C,4-C,3-C-N,7-O 9
C-C-N-C-C-C-C-C,7-C,6-C-C,10-C 12
C-C-C-C-C,4-C,3-C-C,7-N 9
N-C-C-C-C-C,5-C,4-C-C 9
C-C-C-C-C-C,4-C-C,7-C,5-N,1-6 10
C-N-C-C-C-C,4-C-C 8
C-C-C-C-C,4-C,3-C-C,7-C 9
C8-C8-C8-C8-C8-C8-C8-C8-C8-C8-C8-N4 12
C8-C8-C8-C8-C8-C8-C8-N4,2-7 8
C8-C8-C8-C8-C8-C8-C8-C8-C8-C8,7-C8-C8-10 12
C8y-C8y-C8y-C8y-C5x-N1x,5-O5x 7
C-C-C-C-C-C-C-C-N,1-9,2-

In [12]:
kyoutuu2 = []
for vecstr in strlist[0]:
    no_counter = 0
    for vec in strlist:
        if vecstr not in vec.values:
            no_counter += 1
    else:
        if no_counter < len(strlist)/2:
            kyoutuu2.append(vecstr)

In [13]:
print(len(kyoutuu2))

467


In [14]:
import re
temp2 = []
for i in kyoutuu2:
    temp3 = i.split("-")
    length = len(temp3)
    for i in re.findall("-[0-9]", "-".join(temp3)):
        length -= 1
    temp2.append((length, temp3))
    
for group in temp2:
    if group[0] > 6:
        print("-".join(group[1]), group[0])

C-C-C-C-C-C-C-C-C-C,7-C-C-10 12
C-C-C-C-C-C-C-C-C-C-C-N 12
C-C-C-C-C-C-C-C-C,6-C-C-9 11
C-C-C-C-C-C,5-C 7
C-C-C-C-C-C-C-C,7-C 9
C-C-C-C-C-C-C,6-C 8
C-C-C-C-C-C-C-C-N,1-6,1-9 9
C-C-C-C-C,4-C,3-N-C,7-C 9
C-C-C-C-C-C-C-C-C-C-C,8-C-C-11 13
C-C-C-C-C,4-C,3-C-C 8
C-C-C-C-C-C-C-C-C-C-C-C,9-C-C-12 14
C-N-C-C-C,4-C,3-C-C 8
C8-C8-C8-C8-C8-C8-C8-C8-N4,1-6,1-9 9
C8-C8-C8-C8-C8,4-C8,3-N4-C1,7-C8 9
C-C-C-C-C-C-C-C,5-C-C-8 10
C-C-C-C-C-C,5-C,4-C-C 9
C-N-C-C-C-C-C-C-C-C-C-C,9-C-C-12 14
C-C-C-C-C-C,4-C-C 8
N-C-C-C-C-C-C-C-C-C-C,8-C-C-11 13
C-C-C-C-C-C-C-C-C,7-C-C 11
C-C-C-C-C-C-C-C 8
C-C-C-C-C-C-C,4-C-C-7 9
C-C-C-C-C-C-C-C,6-C-C 10
C-N-C-C-C-C-C-C 8
C-N-C-C-C-C-C-C-C-C,6-C-10 11
C-C-C-C-C-N-C-O-C-N,1-10,2-6 10
C-C-C-C-C,3-C-C,6-C,4-N 9
C-N-C-C-C-C-C,6-C,5-C-C,9-C 11
O-C-C-C-C,4-C,3-C-C,7-C 9
C-N-C-C-C,3-C-C,6-C 8
C-C-N-C-C-C-C-C-C-C-C-C,7-12 12
C-N-C-C-C-C-C-C-C,8-C 10
C-N-C-C-C-C-C-C,6-C-C,5-C-C-C,12-C,3-13 14
C-C-C-N-C-C,5-C,4-C-C,8-C 10
C-N-C-C-C-C,5-C 7
N-C-C-C-C-C-C-C,7-C 9
C-C-C-C-C-C-C-C-C-C,8-C

In [16]:
set(veclist[0].pandas()["type"])

{'atom', 'bond', 'linear', 'ring', 'triplet', 'unit', 'vicinity'}

数に幅をもたせてkcf-sを抜き出してみた。  
どこかで使うかも。